# Understanding functions
We can create functions that have parameters

In [1]:
def hello_person(name):
    """ Says hello to someone """
    return "hello {}".format(name)

hello_person("sam")

'hello sam'

We can also create functions inside other functions. The inner functions will have access to the parameters from the parent. In this example the inner function uses the `message` param from the parent.

In [2]:
def message_to_person(message, name):
    
    def send_message():
        """ Sends message """
        return message
    
    return "{} {}".format(send_message(), name)

message_to_person("Welcome", "Phil")

'Welcome Phil'

# Decorators basics
We can decorate a function so that some things happen "around" a function.

In [3]:
def decorate_h1(func):
    """ Decorates a function in order to add <h1> tags """
    
    def make_h1(text):
        return "<h1>{}</h1>".format(func(text))
    
    return make_h1
    
hello_person_h1 = decorate_h1(hello_person)
hello_person_h1("John")

'<h1>hello John</h1>'

Here we have decorated the `hello_world` function to add `h1` tags around it.
The idea is that we can make some steps before and after the function we are decorating.

In [4]:
from time import time, sleep

In [5]:
def timeit(func):
    """ Timing decorator """
    
    def timed(*args):
        """ Prints the execution time of a function """
        t0 = time()
        result = func(*args)
        print("Done in {:.2f} seconds".format(time() - t0))    
        return result
    
    return timed

In [6]:
def wait_3_seconds():
    sleep(3)
    
wait_3_seconds()

In [7]:
wait_3_seconds_and_print = timeit(wait_3_seconds)

wait_3_seconds_and_print()

Done in 3.00 seconds


# Using real decorators
Now that we understand the idea behind decorators, we can rewrite the examples using real decorators

In [8]:
@decorate_h1
def hello_person_with_h1(name):
    """ Says hello to someone """

    return "hello {}".format(name)

hello_person_with_h1("Anna")

'<h1>hello Anna</h1>'

In [9]:
@timeit
def wait_3_seconds_with_print():
    sleep(3)

wait_3_seconds_with_print()

Done in 3.00 seconds


And we can do it with one line. This is equivalent to the above example

In [10]:
sleep_and_print = timeit(sleep)

sleep_and_print(2)

Done in 2.00 seconds


And we can combine them both

In [11]:
@timeit
@decorate_h1
def wait_and_say_hello(name):
    """ Says hello to someone """
    sleep(3)
    return "hello {}".format(name)

wait_and_say_hello("Turtle")

Done in 3.00 seconds


'<h1>hello Turtle</h1>'

So the idea is to do things before or after the function

In [12]:
def printer(func):
    """ Printer decorator """
    
    def prints():
        """ Prints some things """
        print("Before function")
        result = func()
        print("After function")    
        return result
    
    return prints

@printer
def hello():
    print("hello world")

hello()

Before function
hello world
After function


# Passing arguments to decorators

In [13]:
def printer_with_params(before="Before function", after="After function"):
    """ Wrapper around decorator """
    
    def printer(func):
        """ Printer decorator """

        def prints():
            """ Prints some things """
            print(before)
            result = func()
            print(after)    
            return result

        return prints
    
    return printer

@printer_with_params(before="Argh!")
def hello_argh():
    print("hello world")
    
hello_argh()

Argh!
hello world
After function


# Some cool examples
### supertimeit
This is a decorator that takes one outputting function and displays the execution time. By default it will use the `print` function.

In [14]:
def supertimeit(output_func=print):
    """ Allows to use custom output functions (like print, log.info...)"""

    def timeit_decorator(func):
        """ Timing decorator """
    
        def timed_execution(*args):
            """ Outputs the execution time of a function """
            t0 = time()
            result = func(*args)
            output_func("Done in {:.2f} seconds".format(time() - t0))    
            return result

        return timed_execution
    
    return timeit_decorator

In [15]:
@supertimeit() # Since it can take params, we need the '()'
def wait_2_seconds_and_print():
    sleep(2)

wait_2_seconds_and_print()

Done in 2.00 seconds


Let's import the logging library and try it

In [16]:
import logging
logging.warning('This is an info message')

Now let's use the `supertimeit` decorator with the `logging.warning` function

In [17]:
@supertimeit(logging.warning)
def wait_2_seconds_and_log():
    sleep(2)

wait_2_seconds_and_log()

## infallible
This decorator will prevent any function from crashing and it will show the details of any `Exception` occurred.

In [18]:
def infallible(output_func=print):
    """ Allows to use custom output functions (like print, log.info...)"""

    def infallible_decorator(func):
        """ Timing decorator """
    
        def infallible_execution(*args):
            """ Outputs the execution time of a function """
            
            try:
                return func(*args)
            except Exception as e:
                output_func("Error in '{}': {}".format(func.__name__, e))

        return infallible_execution
    
    return infallible_decorator

In [19]:
@infallible()
def wrong_math(x):
    """ This divides a number by 0 """
    return x/0

In [20]:
wrong_math(1)

Error in 'wrong_math': division by zero


And of course we can combine them both

In [21]:
@supertimeit()
@infallible(logging.error)
def unuseful_function(x):
    """ It will wait some time and then it will divide a number by 0 """

    sleep(2)
    return x/0

In [22]:
unuseful_function(10)

ERROR:root:Error in 'unuseful_function': division by zero


Done in 2.00 seconds
